# modules

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from keras import optimizers
import keras.layers.advanced_activations
import scipy
import random
import matplotlib.pyplot as plt  
from scipy.stats import norm  
import keras
from keras.models import Sequential
from keras.initializers import VarianceScaling,RandomNormal
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, concatenate, MaxPooling2D, AveragePooling2D
from keras.layers import UpSampling2D, Reshape, Lambda, Flatten, Activation,Concatenate
from keras.models import Model  
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras import backend as K  
from keras import objectives  
from keras.utils.vis_utils import plot_model  
from keras.utils import np_utils, generic_utils
import sys 
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib inline
from sklearn import manifold, datasets,cluster
from sklearn.utils import check_random_state
from sklearn import metrics


/home/hao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# load training data and testing data

In [2]:
path = os.listdir('/home/hao/DATAS/Salt/train/images')
train_image = []
train_mask = []
for i in range(len(path)):
    path_now = path[i]
    a = Image.open('/home/hao/DATAS/Salt/train/images/'+str(path_now))
    a = np.asarray(a)
    train_image.append(a)
    b = Image.open('/home/hao/DATAS/Salt/train/masks/'+str(path_now))
    b = np.asarray(b)
    train_mask.append(b)
train_image = np.asarray(train_image)
train_mask = np.asarray(train_mask)

train_image = train_image/np.max(train_image)
train_mask = train_mask/np.max(train_mask)

In [3]:
path_test = os.listdir('/home/hao/DATAS/Salt/test/images')
test_image = []
for i in range(len(path_test)):
    path_now = path_test[i]
    c = Image.open('/home/hao/DATAS/Salt/test/images/' + str(path_now))
    c = np.asarray(c)
    test_image.append(c)
test_image = np.asarray(test_image)
test_image = test_image/np.max(test_image)

In [4]:
print(train_mask.shape)
print(train_image.shape)

(4000, 101, 101)
(4000, 101, 101, 3)


In [5]:
batch_size =20
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = 96*96
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:

input_img = Input(shape=(96,96,3))

conv_1 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(input_img)
conv_1 = Activation('relu')(conv_1)
conv_1 = BatchNormalization()(conv_1)
maxpool_1 = MaxPooling2D((2, 2),padding='same')(conv_1)

conv_2 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(maxpool_1)
conv_2 = Activation('relu')(conv_2)
conv_2 = BatchNormalization()(conv_2)
maxpool_2 = MaxPooling2D((2, 2),  padding='same')(conv_2)

conv_3 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_2)
conv_3 = Activation(LRelu)(conv_3)
conv_3 = BatchNormalization()(conv_3)
maxpool_3 = MaxPooling2D((2, 2),  padding='same')(conv_3)

conv_4 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_3)
conv_4 = Activation(LRelu)(conv_4)
conv_4 = BatchNormalization()(conv_4)
maxpool_4 = MaxPooling2D((2, 2),  padding='same')(conv_4)

conv_5 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(maxpool_4)
maxpool_5 = MaxPooling2D((2, 2),  padding='same')(conv_5)


#x = Conv2D(5, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(x)
#x = MaxPooling2D((2, 2),  padding='same')(x)

visual = Flatten()(maxpool_5)
h_1 = Dense(intermediate_dim, activation='relu')(visual)#relu?
encoded = Dense(latent_dim, activation='tanh')(h_1)# relu?


h_3 = Dense(intermediate_dim,activation=LRelu)(encoded)#for AE

h_4 = Dense(20*3*3,activation=LRelu)(h_3)
h_5 = Reshape((3,3,20))(h_4)


conv_6 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(h_5)
upsample_6 = UpSampling2D((2, 2))(conv_6)

upsample_7 = Concatenate()([upsample_6,conv_5])
conv_7 = Conv2D(10, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_6)
upsample_7 = UpSampling2D((2, 2))(conv_7)

upsample_7 = Concatenate()([upsample_7,conv_4])
conv_8 = Conv2D(10, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_7)
upsample_8 = UpSampling2D((2, 2))(conv_8)

upsample_8 = Concatenate()([upsample_8,conv_3])
conv_9 = Conv2D(10, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_8)
upsample_9 = UpSampling2D((2, 2))(conv_9)

#upsample_9 = Concatenate()([upsample_9,conv_2])
conv_10 = Conv2D(5,  (3, 3), activation='relu',padding='same',kernel_initializer='normal')(upsample_9)
upsample_10 = UpSampling2D((2, 2))(conv_10)

#upsample_10 = Concatenate()([upsample_10,conv_1])
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(upsample_10)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
autoencoder = Model(inputs=input_img, outputs=decoded)
autoencoder.compile(optimizer=adam, loss=dice_loss)

In [ ]:
#CheckPoint = keras.callbacks.ModelCheckpoint("Model_keras.h5", monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
autoencoder.fit(train_image_96, train_mask_96,
        shuffle=True,
        epochs=50,
        batch_size=5,
        validation_split = 0.1,callbacks=[EarlyStopping,checkpoint])

In [ ]:
autoencoder.load_weights('.h5')

In [ ]:
test_pred = autoencoder.predict(test_image[:,5:,5:,:])
test_pred_zeroone = np.round(test_pred)


In [6]:
def aug101(test_pred_now):
    up = test_pred_now[:5,:,0]
    half = np.vstack((up,test_pred_now[:,:,0]))
    left = half[:,:5]
    final = np.hstack((left,half))
    final = np.reshape(final,[101,101,1])
    return final

def to_line(pic):
    line = []
    if len(np.shape(pic))==3:
        pic = pic[:,:,0]
    for i in range(101):
        for j in range(101):
            line.append(pic[j,i])
    return line

def to_final_result(line):
    i = 1
    starts = []
    lenths = []
    while i<=len(line):
        j = 0
        if line[i-1] == 1:
            starts.append(i)
            j = 1
            if i<len(line):
                while line[i-1+j]==1:
                    j+=1
                    if i-1+j>= len(line):
                        break
            lenths.append(j)
        i += j+1
    comma_split = []
    for i in range(len(starts)):
        comma_split.append(starts.pop(0))
        comma_split.append(lenths.pop(0))
    return comma_split


In [ ]:
test_pred_zeroone_101 = []
for i in range(len(test_pred_zeroone)):
    test_pred_zeroone_101.append(aug101(test_pred_zeroone[i]))
test_pred_zeroone_101 = np.asarray(test_pred_zeroone_101)

In [ ]:
output = []
for i in range(18000):
    if i%1000 == 0:
        print(i)
    temp = test_pred_zeroone_101[i]
    l = to_final_result(to_line(temp))
    k = (" ".join(str(i) for i in l))
    output.append((path_test[i][:-4],k))
output = np.asarray(output)

In [ ]:
np.savetxt('output2.txt',output ,fmt = '%s',delimiter = ',')